In [1]:
import pandas as pd
import numpy as np
import glob
import re

### Extract affordable housing financing data from Virginia PDFs

In [2]:
#overwrite pandas to let it show full document without truncation so I can manually
#look for where to index in
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

/Users/quinnunderriner/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  


In [13]:
#filepath to PDFs
filepath = '/Users/quinnunderriner/Desktop/Work/Georgia 2019 Applications/'
# create a list of filepaths 
files = glob.glob(filepath+"*.xlsx", recursive = True)

In [384]:
def extract_from_georgia_xlsx(files):
    """
    Inputs:
        files (list) - strings of path to excel files
    
    This function looks for the first instance of the phrase "Construction Financing" and 
    then indexes through the file to pull out relevent financial info by section. 
    """
    final_df = pd.DataFrame()
    for file in files:
        #print("starting file: ", file)
        file_name = re.split("/", file)[-1]
        file_name = file_name[0:-5]
        
        df = pd.read_excel(file, sheet_name='Part III-Sources of Funds')
        #need to rename columns to numbers to make indexing easier 
        df = df.rename(columns={x:y for x,y in zip(df.columns,range(0,len(df.columns)))})
        try:
            #construction financing section 
            row1 = df[df[1] == 'CONSTRUCTION FINANCING'].index[0]
            df1 = df[row1:row1+14].dropna(subset=[11]).dropna(how='all', axis=1)
            df1.columns = df1.iloc[0]
            df1 = df1[1:]
            df1["File Name"] = file_name
            df1["Funding Type"] = "Construction Financing"
            df1 = df1.drop(["DCA COMMENTS - DCA Use Only"],axis=1,)
            df1 = df1[pd.notnull(df1[' Name of Financing Entity'])]#drop col if null name for financing entity
            df1 = df1.loc[:, df1.columns.notnull()]#drop col if name is null 
            df1 = df1.rename(columns={"Term (In Months)":"Term (Months)",
                                      "Amount ($)":"Amount of Funds",
                                     ' Name of Financing Entity':'Name of Financing Entity'})
        except Exception as e:
            print("no construction financing ",file_name, e)

        try:    
            #permanent financing section 
            row2 = df[df[1] == 'PERMANENT FINANCING'].index[0]
            df2 = df[row2:row2+10].dropna(subset=[8]).dropna(how='all', axis=1)
            df2 = df2.rename(columns=df2.iloc[0]).drop(df2.index[0])#make first row the header
            df2 = df2.loc[:,~df2.columns.duplicated()]#removed duplicate col names
            df2 = df2.loc[:, df2.columns.notnull()]#drop col if name is null 
            df2 = df2.drop(["DCA COMMENTS - DCA Use Only","Loan Type"],axis=1,)
            df2["(Years)"] = df2["(Years)"]*12 #change years to months
            df2 = df2.rename(columns={"Principal Amount ($)":'Amount of Funds',
                                      "Int Rate":"Effective Interest Rate",
                                      "(Years)":"Term (Months)",
                                     ' Name of Financing Entity':'Name of Financing Entity'})
            df2["File Name"] = file_name
            df2["Funding Type"] = "Permanent Financing"
            final_df1 = pd.concat([df1, df2])
        except Exception as e:
            print("no perm  financing 1 ",file_name, e)
        try:
            #permanent financing section subsection 
            row3 = df[df[1] == 'Federal Grant'].index[1]
            df3 = df.loc[row3:row3+11, :8].dropna(subset=[8]).dropna(how='all', axis=1)
            df3 = df3.rename(columns = {1:"Financing Type",4:"Name of Financing Entity",
                                        8:"Amount of Funds"})
            #df3 = df3.drop([2], axis=1)
            df3 = df3[df3['Name of Financing Entity'].notna()]
            df3['File Name'] = file_name
            df3['Funding Type'] = 'Permanent Financing'
        
        except Exception as e:
            print("no perm  financing 2 ",file_name, e)
        
        try:
            final_df2 = final_df1.append(df3, ignore_index=True)
        except Exception as e:
            print("final concat failed ",file_name, e)
        
        
        #final_df2 = final_df2.values.tolist()
        final_df = final_df.append(final_df2, ignore_index=True)
    final_df = final_df.drop([2], axis=1)
    return final_df

In [387]:
#need to manually handle these edge cases. The final three errors are just that it was a blank page. 
df = extract_from_georgia_xlsx(files)

no perm  financing 2  2019-063havensatlantagoracore 'Name of Financing Entity'
no perm  financing 2  2019-073arborskingsldgoracore 'Name of Financing Entity'


/Users/quinnunderriner/anaconda3/lib/python3.7/site-packages/pandas/core/computation/expressions.py:68: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return op(a, b)
/Users/quinnunderriner/anaconda3/lib/python3.7/site-packages/pandas/core/computation/expressions.py:68: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return op(a, b)
/Users/quinnunderriner/anaconda3/lib/python3.7/site-packages/pandas/core/computation/expressions.py:68: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return op(a, b)


no construction financing  2019-018villageon5thgoraapp index 0 is out of bounds for axis 0 with size 0
no perm  financing 1  2019-018villageon5thgoraapp index 0 is out of bounds for axis 0 with size 0
no perm  financing 2  2019-018villageon5thgoraapp index 1 is out of bounds for axis 0 with size 0


In [390]:
#write to csv
df.to_csv("Georgia_scraped_financing_info.csv")

In [45]:
df = pd.read_csv("Georgia_scraped_financing_info.csv")

In [ ]:
#filter for only accepted 9% applications, data found here: https://www.dca.ga.gov/node/3814 although the names don't 
#match the file names perfectly and are super messy 

In [26]:
#need to standardize name column so files can be filtered. Need to drop date from front end, 
#and goracore from the back end. index in 8, back -7. also drop 2s, because inconsistent with filter list 
names_in_file = df["File Name"].str[8:-7].unique()

In [29]:
sorted(list(names_in_file))

['_24east',
 '_idorag',
 'abbgntnormewdpkg',
 'abbgtnsoundg',
 'abbngtnsoundg',
 'anthm2',
 'arborskingsldg',
 'belleviewoaksg',
 'brennanplcg',
 'canaancrssgg',
 'capitallndgg',
 'chateauhill',
 'chattahoocheeptg',
 'chelseaparkth_g',
 'covecartersvilleg',
 'creeksideg',
 'crowvllyhtsg',
 'dogwoodtrail2g',
 'dullespark2g',
 'elmstreetg',
 'flatsaberdeen',
 'flatssamlaneg',
 'flintridgedr_g',
 'flintridgeg',
 'groveparkg',
 'grovewoodford',
 'haddocklndg',
 'haddocklndgg',
 'harperwdsg',
 'havensatlantag',
 'havenwdsprgsdr_g',
 'havenwoodgdnsg',
 'hearthsidesmyrnag',
 'heritageg',
 'jesupcommonsg',
 'kingsbaygdns',
 'kingsbaygdnsg',
 'kingsbrook',
 'kingsmlllndgs',
 'kingsmlllndgscdbg_g',
 'legacyg',
 'legacywaltontrlg',
 'liveoaklndg2',
 'millvillageg',
 'newnancrossg',
 'northsidehtsg',
 'northsidesrvlgg',
 'nrthlakecommns',
 'nwncrsg',
 'oaklandg',
 'peaksclaytong',
 'peaksdawsonvilleg',
 'perryplaceg',
 'pineviewtraceg',
 'redwoodtrailsg',
 'revere',
 'riverpte2g',
 'sandyrunctg',


In [31]:
sorted(filter1)

['24 East',
 'Abbgtn Ormewd Prk',
 'Anthem II',
 'Canaan Crossing',
 'Cove Cartersville',
 'Creekside',
 'Crow Valley Hts',
 'Flats Sam Lane',
 'Flint Ridge',
 'Harper Woods',
 'Haven S Alanta',
 'Havenwood Gdns',
 'Hearthside Smyrna',
 'Legacy Walton Trl',
 'Live Oak Lndg II',
 'Mill Vllg',
 'Newnan Crossing',
 'Northside Sr Vllg',
 'Pineview Trace',
 'Sterling Hts',
 'Sutton Terrace',
 'Symphony',
 'Thrive Swt Auburn',
 'Veranda Assembly',
 'Vllg Chickamauga 2',
 'Vllg on 5th',
 'WR Redevlpmt II',
 'Walton Harbor',
 'Waring',
 'Westover Sr Vllg',
 'Wimberly Manor',
 'Woodlawn Sr Vllg',
 'Woodstone II']

In [3]:
filter1 = pd.read_excel("2019_funding_cycle_selection_revised12162019 (2).xlsx")

In [5]:
filter1 = filter1["Unnamed: 3"].dropna().to_list()[1:35]#indexed in based just on manually counting 
#because they have funded and unfunded projects in the same col for some reason 
del filter1[15] # remove number 15 because its just a number not a project name 

In [24]:
filter1 = [x.strip(' ') for x in filter1]#filter out the whitespace 

In [35]:
#now want to filter for just selected projects
#data on projects selected found here: https://www.dca.ga.gov/node/3814
#gonna manually translate between these two, far too many missing letters 
accepted_apps = ['_24east',
'abbgntnormewdpkg',
'anthm2',
'canaancrssgg',
'creeksideg',
'crowvllyhtsg',
'flatssamlaneg',
 'flintridgedr_g',
 'flintridgeg',
 'harperwdsg',
 'havensatlantag',
 'hearthsidesmyrnag',
'legacywaltontrlg',
'liveoaklndg2',
'millvillageg',
'newnancrossg',
'northsidesrvlgg',
 'pineviewtraceg',
 'sterlinghtsg',
  'suttonterrace',
'symphony',
 'thriveswtauburng',
  'verandaassemblyg',
  'villgchickamauga2g',
  'waltonharborg',
  'waring',
  'westoversr',
  'woodlawnsrvllgg',
 'woodstone2g',
 'wrredev2g']

In [46]:
df["File Name"] = df["File Name"].str[8:-7]



,Financing Type,Name of Financing Entity,Amount of Funds,Effective Interest Rate,Term (Months),File Name,Funding Type
29,Mortgage A,Bank OZK,2744584.43,0.0594,12.0,waring,Construction Financing
30,Federal Housing Credit Equity,Raymond James Federal Equity,466848,NaN,NaN,waring,Construction Financing
31,State Housing Credit Equity,Raymond James State Equity,280772,NaN,NaN,waring,Construction Financing
32,Deferred Devlpr Fee,DASH Development/CRN Development,11779,0.0000,12.0,waring,Permanent Financing
33,Federal Housing Credit Equity,Raymond James Federal Equity,2651735,NaN,NaN,waring,Permanent Financing
...,...,...,...,...,...,...,...
590,Federal Housing Credit Equity,Raymond James,2945885,NaN,NaN,woodlawnsrvllgg,Construction Financing
591,State Housing Credit Equity,Raymond James,1386317,NaN,NaN,woodlawnsrvllgg,Construction Financing
592,Deferred Devlpr Fee,deferred developer fee,8118,0.0000,12.0,woodlawnsrvllgg,Permanent Financing
593,Federal Housing Credit Equity,Raymond James,3187500,NaN,NaN,woodlawnsrvllgg,Permanent Financing


In [49]:
len(df)

238

In [48]:
df = df[df["File Name"].isin(accepted_apps)]

In [90]:
df = df[df["Financing Type"] != "Total Permanent Financing:"] #remove total financing mistake

In [92]:
df.to_csv("Georgia_approved_9%_applications.csv")

In [ ]:
#now trying to auto_label some of the funding types 

In [91]:
df["Financing Type"].value_counts()

Federal Housing Credit Equity     61
State Housing Credit Equity       61
Mortgage A                        30
Deferred Devlpr Fee               25
Mortgage A (Lien Position 1)      24
Mortgage B                        15
Mortgage B (Lien Position 2)       8
State, Local, or Private Grant     4
Mortgage C                         2
Mortgage C (Lien Position 3)       2
Federal Grant                      2
Other:                             1
Deferred Developer Fees            1
Other Type (specify)               1
Name: Financing Type, dtype: int64

In [74]:
#create subcategories for types of financing, below code applies these and then creates a new column 
bank_loan = ['Mortgage A','Mortgage B','Mortgage C','Mortgage A (Lien Position 1)',
             "Mortgage B (Lien Position 2)","Mortgage C (Lien Position 3)"]
deferred_dev = ["Deferred Devlpr Fee","Deferred Developer Fees"]
state_credit = ["State Housing Credit Equity"] #these are all listed as the banks who are taking the other side of the tax credit 
federal_credit = ["Federal Housing Credit Equity"]
local_grant = ["State, Local, or Private Grant"] #spot checked these and 2 are from City of Gainesville - CDBG and two are from Macon Water Authority
public_private_partnership = ["Other Type (specify)","Other:"] # both are a partnership between the city of atlanta and united way https://partnersforhome.org/wp-content/uploads/2018/11/PSHDevelopment_ppt_111518.pdf
fed_grant = ["Federal Grant"]

In [77]:
conditions = [df["Financing Type"].isin(bank_loan),
    df["Financing Type"].isin(deferred_dev),
    df["Financing Type"].isin(state_credit),
    df["Financing Type"].isin(federal_credit),
    df["Financing Type"].isin(local_grant),
    df["Financing Type"].isin(public_private_partnership),
    df["Financing Type"].isin(fed_grant)]



In [80]:
outputs = ['Bank Loan',"Deferred Developer Fee","State Housing Credit Equity",
    "Federal Housing Credit Equity","Local Grant","Public Private Partnership","Federal Grant"]

In [85]:
df["Financing_sub_cat"] = np.select(conditions, outputs, 'Other')


In [97]:
df.drop(["Unnamed: 0"],axis=1).to_csv("Georgia_scraped_financing_info_Awarded_Only.csv")

In [99]:
df["Financing_sub_cat"].unique()

array(['Bank Loan', 'Federal Housing Credit Equity',
       'State Housing Credit Equity', 'Deferred Developer Fee',
       'Federal Grant', 'Public Private Partnership', 'Local Grant'],
      dtype=object)

### pt2 - gather extra info 

In [4]:
df = pd.read_csv("Georgia_approved_9%_applications.csv") 

In [38]:
test_files = files[0:10]
file = files[0]


In [105]:
def extract_from_georgia_xlsx(files):
    """
    Inputs:
        files (list) - strings of path to excel files
    
    This function looks for the first instance of the phrase "Construction Financing" and 
    then indexes through the file to pull out relevent financial info by section. 
    """
    cols = ['Project Name', 'Zip Code', 'Project Type', "Total Units",
                                 "Total Square Footage","Census Tract","Total Project Cost"]
    final_df = pd.DataFrame(columns=cols)
    df = []
    for file in files:
        #print("starting file: ", file)
        file_name = re.split("/", file)[-1]
        file_name = file_name[0:-5]
        
        try:
            project_information = pd.read_excel(file, sheet_name='Part I-Project Information')
            costs = pd.read_excel(file, sheet_name='Part IV-A-Uses of Funds')
        except Exception as e:
            print("can read data",file_name, e)
        try:
        #need to rename columns to numbers to make indexing easier 
            project_information = project_information.rename(columns={x:y for x,y in zip(project_information.columns,range(0,len(project_information.columns)))})
            costs = costs.rename(columns={x:y for x,y in zip(costs.columns,range(0,len(costs.columns)))})
        except Exception as e:
            print("can change headings",file_name, e)
            
        try:
            #new_row = {'Project Name':file_name,"Zip Code":project_information[9][36],
                       #'Project Type':project_information[7][80],"Total Units":project_information[7][70],
                      #"Total Square Footage":project_information[15][73],"Census Tract":project_information[14][36],
                      #"Total Project Cost":costs[6][130]}
            new_row = [file_name, project_information[9][36],project_information[7][80],project_information[7][70],
                      project_information[15][73],project_information[14][36],'{:f}'.format(costs[6][130])]
            
            
        except Exception as e:
            print("can add new info",file_name, e)
            
        try:
            zipped = zip(cols, new_row)
            add_from_dict = dict(zipped)
            df.append(add_from_dict)
        
        except Exception as e:
            print("can add new info",file_name, e)
        
        print(file_name,"i am project name")
        print(project_information[9][36],"i am zip code")
        print(project_information[7][80],"I am project type")
        print(project_information[7][70],"I am total units")
        print(project_information[15][73],"I am total square footage")
        print(project_information[14][36],"I am census tract")
        print(costs[6][130],"I am total project cost")
        
    final_df = final_df.append(df, True)   
    return final_df

In [ ]:
# function call to scape new data 
final = extract_from_georgia_xlsx(files)

In [110]:
final.to_csv("Georgia_additional_info.csv")


In [136]:
final = pd.read_csv("Georgia_additional_info.csv")

In [ ]:
#some final data cleaning for the merge 
df = df.drop(columns=["Unnamed: 0","Unnamed: 0.1"]) #drop index columns that got made earlier 
final["Project Name"] = final["Project Name"].str[8:-7] # strip project name to match for merging 
final = final.rename(columns={"Project Name":"File Name"}) #make columns same name 

#merge new extra info onto already scraped data
merged = df.merge(final,on="File Name")

#bit of final data cleaning, drop some rows, correct types etc 

merged["Amount of Funds"] = merged["Amount of Funds"].round(2)
merged["Total Project Cost"] = merged["Total Project Cost"].round(2)
merged["Zip Code"] = merged["Zip Code"].astype(str).str[:-4].astype(np.int64) #make it 5 digit zip instead of 9
merged = merged.drop(columns=["Unnamed: 0"])

In [164]:
merged.to_csv("Georgia_Final_Data.csv")

In [163]:
ls

2019_funding_cycle_selection_revised12162019 (2).xlsx
Georgia_Financials_PDF_Extraction-Copy1.ipynb
Georgia_Financials_PDF_Extraction.ipynb
Georgia_Financials_PDF_Extraction_V2_MoreFields.ipynb
Georgia_approved_9%_applications.csv
Georgia_scraped_financing_info.csv
Georgia_scraped_financing_info_Awarded_Only.csv
